In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
poems = np.load('normalized_poem_text.npy')
raw_text = '\n'.join(poems)
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Number of characters in the datatset: ", n_chars)
print("unique charachter count: ", n_vocab)

In [ ]:
seq_length = 42 # play with this number
step_size = 1 # and this number
dataX = [] # list of sentences of length seq_length
dataY = [] # list of charachters following that sentence
for i in range(0, n_chars - seq_length, step_size):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_seq = len(dataX)
print("number of sequences: ", n_seq)

In [ ]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_seq, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# build the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# define the checkpoint
w_path="weights/weights-epoch-{epoch%10}.hdf5"
checkpoint = ModelCheckpoint(w_path, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')

In [ ]:
model.fit(X, y, epochs=50, batch_size=128, callbacks=[checkpoint])

## Generate Text

In [ ]:

chars = # TODO: save char list externally and load it here
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
filename = ''

model = Sequential()
model.add(Dropout(0.3, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

model.load_weights(filename)
optimizer = RMSprop(lr=0.01) # TODO: experiment with dropout and learning rates
model.compile(loss='categorical_crossentropy', optimizer= optimizer)

In [ ]:
seed = ''
sequence = [char_to_int[char] for char in seed]
gen_text = ''
print('Seed:\n')
print(seed)
# generate characters
for i in range(140):
    x = np.reshape(sequence, (1, len(sequence), 1))
    x = x / len(chars)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction) # returns the index of the predicted char
    gen_text += int_to_char[index]
    sequence.append(index)
    sequence = sequence[1:len(sequence)]

print(gen_text)
print('Done.')